In [9]:
import pandas as pd
import datetime as dt
import numpy as np
import requests
import json
import math

In [10]:
df = pd.read_csv("sensor_data.csv")
df = df[['IdSensore', 'Data', 'Valore']]
df['Data'] = pd.to_datetime(df['Data'], format="%d/%m/%Y %I:%M:%S %p", errors="coerce") #01/01/2023 12:00:00 AM

In [19]:
df

,IdSensore,Data,Valore
0,5504,2023-01-01 00:00:00,42.1
1,5504,2023-01-01 01:00:00,41.1
2,5504,2023-01-01 02:00:00,39.7
3,5504,2023-01-01 03:00:00,38.6
4,5504,2023-01-01 04:00:00,36.9
...,...,...,...
2825911,5845,2024-01-21 20:00:00,1.2
2825912,5845,2024-01-21 21:00:00,1.2
2825913,5845,2024-01-21 22:00:00,1.4
2825914,5845,2024-01-21 23:00:00,1.3


In [17]:
complete_air_quality_stations = pd.read_csv("complete_air_quality_stations.csv")

In [24]:
# Nulling stations that do not exist
missing_stations_df = df.copy()
for i, sensor in complete_air_quality_stations.iterrows():
    missing_stations_df = missing_stations_df.loc[missing_stations_df["IdSensore"] != sensor.IdSensore]
    
missing_stations_df



,IdSensore,Data,Valore
991,20565,2023-12-21 00:00:00,55.0
1004,20564,2023-12-21 00:00:00,21.0
1897,20525,2024-01-06 01:00:00,-9999.0
1898,20525,2024-01-06 02:00:00,-9999.0
1899,20525,2024-01-06 03:00:00,-9999.0
...,...,...,...
2824592,20525,2024-01-18 20:00:00,-9999.0
2824593,20525,2024-01-18 21:00:00,-9999.0
2824594,20525,2024-01-18 22:00:00,-9999.0
2824595,20525,2024-01-18 23:00:00,-9999.0


In [27]:
missing_stations_df.IdSensore.unique()
missing_sensors = list(missing_stations_df.IdSensore.unique())
missing_sensors

[20565, 20564, 20525]

In [29]:
# Remove observations of stations that do not exist
for sensor in missing_sensors:
    print(sensor)
    df = df.loc[df["IdSensore"] != sensor]

df

20565
20564
20525


,IdSensore,Data,Valore
0,5504,2023-01-01 00:00:00,42.1
1,5504,2023-01-01 01:00:00,41.1
2,5504,2023-01-01 02:00:00,39.7
3,5504,2023-01-01 03:00:00,38.6
4,5504,2023-01-01 04:00:00,36.9
...,...,...,...
2825911,5845,2024-01-21 20:00:00,1.2
2825912,5845,2024-01-21 21:00:00,1.2
2825913,5845,2024-01-21 22:00:00,1.4
2825914,5845,2024-01-21 23:00:00,1.3


In [36]:
def arrange_timestamp(date_time):
    return str(date_time)

In [37]:
base_url_measurements = "http://127.0.0.1:8000/api/collections/AirQualityMeasurement/"

batch_size = 1000
batches = math.ceil(len(df) / batch_size)
print(f"Number of batches: {batches}")

for i in range(0, batches):
    body = {
        "bulk": True,
        "items": []
    }
    
    i_start = i * batch_size
    i_end = len(df) if len(df) <= (batch_size * (i+1)) else (batch_size * (i+1))
    
    print(f'from {i_start} to {i_end}')
    timeseries_subset = df.iloc[i_start:i_end]
    timeseries_subset = timeseries_subset[['IdSensore','Data','Valore']]
    timeseries_subset['Data'] = timeseries_subset['Data'].apply(arrange_timestamp)
    timeseries_subset = timeseries_subset.replace({np.nan: None})
    
    for i, sensor in timeseries_subset.iterrows():
        item = { "sensor_id_id": sensor.IdSensore, "date": sensor.Data, "value": sensor.Valore }
        body["items"].append(item)
        
    body_json = json.dumps(body)
    body_json
    req = requests.post(base_url_measurements, body_json)
    if req.status_code == 500:
        print(req)
    print(f'request {i} - {req.status_code}')

Number of batches: 2817
from 0 to 1000
request 1000 - 200
from 1000 to 2000
request 2025 - 200
from 2000 to 3000
request 3025 - 200
from 3000 to 4000
request 4025 - 200
from 4000 to 5000
request 5025 - 200
from 5000 to 6000
request 6025 - 200
from 6000 to 7000
request 7025 - 200
from 7000 to 8000
request 8025 - 200
from 8000 to 9000
request 9025 - 200
from 9000 to 10000
request 10025 - 200
from 10000 to 11000
request 11025 - 200
from 11000 to 12000
request 12025 - 200
from 12000 to 13000
request 13025 - 200
from 13000 to 14000
request 14025 - 200
from 14000 to 15000
request 15025 - 200
from 15000 to 16000
request 16025 - 200
from 16000 to 17000
request 17025 - 200
from 17000 to 18000
request 18025 - 200
from 18000 to 19000
request 19025 - 200
from 19000 to 20000
request 20025 - 200
from 20000 to 21000
request 21025 - 200
from 21000 to 22000
request 22025 - 200
from 22000 to 23000
request 23025 - 200
from 23000 to 24000
request 24025 - 200
from 24000 to 25000
request 25025 - 200
from 25

In [35]:
body = {
    "bulk": True,
    "items": []
}

timeseries_subset = df.iloc[0:1]
timeseries_subset = timeseries_subset[['IdSensore','Data','Valore']]
timeseries_subset['Data'] = timeseries_subset['Data'].apply(arrange_timestamp)
timeseries_subset = timeseries_subset.replace({np.nan: None})

for i, sensor in timeseries_subset.iterrows():
    item = { "sensor_id_id": sensor.IdSensore, "date": sensor.Data, "value": sensor.Valore }
    body["items"].append(item)
body_json = json.dumps(body)
body_json

'{"bulk": true, "items": [{"sensor_id_id": 5504, "date": "2023-01-01 00:00:00", "value": 42.1}]}'

In [55]:
count_elements = 2816286

size_b = 431521792
print(f'{size_b / count_elements} bytes per element in table')
print()

size_id = 22530288
print(f'{size_id / count_elements} bytes per element in column id')

size_date = 22530288
print(f'{size_date / count_elements} bytes per element in column date')

size_value = 18191588
print(f'{size_value / count_elements} bytes per element in column value')

size_fk = 11265144
print(f'{size_fk / count_elements} bytes per element in column fk')

print()
total_size = size_id + size_date + size_value + size_fk
print(f'{total_size} total size in bytes')
print(f'{total_size / count_elements} bytes per data element')
print(f'{total_size/1000000} MB')
print()

index_size = 83369984
print(f'{index_size / count_elements} bytes per element in index')

153.22371094412998 bytes per element in table

8.0 bytes per element in column id
8.0 bytes per element in column date
6.45942493056458 bytes per element in column value
4.0 bytes per element in column fk

74517308 total size in bytes
26.45942493056458 bytes per data element
74.517308 MB

29.60281164626036 bytes per element in index


In [52]:
total_data_AQ = 2800000 + 14800000 + 37900000 + 42900000
total_data_AQ

est_b_per_element = 154
est_in_db = total_data_AQ * 154
print(f'{est_in_db} bytes')
print(f'{est_in_db / 1000000} MB')
print(f'{est_in_db / 1000000000} GB')

15153600000 bytes
15153.6 MB
15.1536 GB


In [54]:
83369984

4278260.869565218